In [1]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split

from datetime import datetime
import sys
sys.path.insert(1, '/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/util_funcs/')
from pandas_colFuncs import isBtoDstlnu, whichBisSig, customMCmatching, B_ID

In [2]:
now = datetime.now()
print("time at start =", now)

time at start = 2021-10-17 16:14:44.216066


In [3]:
HTCondorRun = str(sys.argv[1])
print("HTCondorRun:",HTCondorRun)

HTCondorRun: -f


In [4]:
sub = "sub02"

In [5]:
save_data = True
save_preprocessedDataframe = True
tmp_data = False
take_subset = False
subset_size = 100000

delete_top_numFSP_evts=True

In [6]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/"

data_subdir = "FULLCMrun_Dstlnu_SHR_BsX_Ablations/"
root_subdir = "SHR_CMvars_run1/"   # "SHR_LeasVars_run2/"

root_path = nfs_path + "SHR_Hc_correctReco_BsX/" + root_subdir # + sub + "/"

In [7]:
merged = "merged_"
if tmp_data:
    merged += "tmp_"

In [8]:
print(root_path + merged)

/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_


In [9]:
fileY4S = uproot.open(root_path + merged + "DXtagDstl.root:variables")
#afsPath = "/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/"
#fileY4S = uproot.open(afsPath + "DXtagDstl.root:variables")

In [10]:
names = ["gammas","electrons","pions","kaons","muons"]
dfs = []
for name in names:
    filename = root_path + merged + "{}.root:variables".format(name)
    #filename = afsPath + "{}.root:variables".format(name)
    print(filename)
    tmpFileFSPs = uproot.open(filename)
    df_tmp = tmpFileFSPs.arrays(library="pd")
    dfs.append(df_tmp)

/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_gammas.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_electrons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_pions.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_kaons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/merged_muons.root:variables


In [11]:
df_FSPs = pd.concat(dfs)

In [12]:
df_Y4S = fileY4S.arrays(library="pd")

In [13]:
print(df_FSPs.shape[0])
print(df_Y4S.shape[0])

9079729
209268


In [14]:
# delete FSPs for which no Y4S file entry was found
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]

In [15]:
df_Y4S = df_Y4S[(df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)]

df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]

In [16]:
print(df_FSPs.shape[0])

9069249


### delete particles which occur more than ones based on uniqueParticleIdentifier

In [17]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(groupsFSPs_uniqParID.sort_values("count"))

In [18]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
print("groupsFSPs_uniqParID.shape[0]:",groupsFSPs_uniqParID.shape[0])
print("df_Y4S.shape[0]:",df_Y4S.shape[0])

df_FSPs.shape[0]: 9069249
groupsFSPs_uniqParID.shape[0]: 6607459
df_Y4S.shape[0]: 209268


In [19]:
# delete particles which occur more than ones (keep first) and if possible keep the one with basf2_used==1
print("df_FSPs[basf2_used].value_counts():",df_FSPs["basf2_used"].value_counts())
df_FSPs = df_FSPs.sort_values("basf2_used",ascending=False).drop_duplicates(subset=("__event__","uniqueParticleIdentifier"), keep='first')
print("df_FSPs[basf2_used].value_counts():",df_FSPs["basf2_used"].value_counts())

df_FSPs[basf2_used].value_counts(): 0.0    4939720
1.0    4129529
Name: basf2_used, dtype: int64
df_FSPs[basf2_used].value_counts(): 1.0    4123635
0.0    2483824
Name: basf2_used, dtype: int64


In [20]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])

df_FSPs.shape[0]: 6607459


## take a sample if used in notebook for faster processing

In [21]:
all_evt_nums = df_FSPs['__event__'].unique()
all_evt_nums.shape[0]

208729

In [22]:
sample_evt_nums = np.random.choice(all_evt_nums, size=subset_size)
sample_evt_nums.shape[0]

100000

In [23]:
if take_subset:
    df_FSPssample = df_FSPs[df_FSPs['__event__'].isin(sample_evt_nums)]
    #df_Y4S=df_Y4Ssample
    df_FSPs=df_FSPssample

In [24]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
print("numEvents:",df_FSPs['__event__'].unique().shape[0])

df_FSPs.shape[0]: 6607459
numEvents: 208729


## save sample 

In [25]:
if take_subset:
    df_FSPs.to_csv(root_path + "df_FSPs_sample_{}_evts.csv".format(subset_size))
    #df_Y4S.to_csv(root_path + "df_Y4S_sample__evts.csv")

In [26]:
load = False
if load:
    df_FSPs = pd.read_csv(root_path + "df_FSPs_sample_{}_evts.csv".format(subset_size))
    #df_Y4S = pd.read_csv(root_path + "df_Y4S_sample10evts.csv")

## filter the wanted D*lnu and Hc isSignal==1 events

In [27]:
df_Y4S['isBtoDstlnu'] = df_Y4S.apply(isBtoDstlnu, axis=1)

In [28]:
df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)].shape[0]

128456

In [29]:
df_Y4S = df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)]

In [30]:
# delete FSPs for which no Y4S file entry is left after filer
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]
df_FSPs.shape[0]

3853047

### add cols with extra info for data prod

In [31]:
df_Y4S['Bsig_uniqParID'] = df_Y4S.apply(whichBisSig, axis=1)

In [32]:
# function to create col with the particles mother B's uniqueParticleIdentifier
df_FSPs['B_ID'] = df_FSPs.apply(B_ID, axis=1)

### print one event

In [33]:
df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)][["__event__",'Bsig_uniqParID','isBtoDstlnu','Hc_isSignalAcceptMissingGamma','Hc_mcPDG','Hc_genMotherPDG','Hc_uniqParID','Hc_genMotherID']][:10]

,__event__,Bsig_uniqParID,isBtoDstlnu,Hc_isSignalAcceptMissingGamma,Hc_mcPDG,Hc_genMotherPDG,Hc_uniqParID,Hc_genMotherID
0,23590230,83886081.0,1,1.0,421.0,423.0,100663296.0,10.0
1,23590504,83886082.0,1,1.0,411.0,-511.0,100663296.0,1.0
3,23596654,83886081.0,1,1.0,-421.0,-413.0,100663296.0,7.0
4,23596983,83886082.0,1,1.0,421.0,413.0,100663296.0,3.0
5,22185250,83886081.0,1,1.0,-411.0,511.0,100663296.0,2.0
7,22188982,83886082.0,1,1.0,-421.0,-413.0,100663296.0,6.0
8,22189313,83886081.0,1,1.0,413.0,20413.0,100663296.0,6.0
11,26685075,83886081.0,1,1.0,421.0,423.0,100663296.0,6.0
13,26687938,83886081.0,1,1.0,-431.0,-511.0,100663296.0,2.0
15,26690307,83886081.0,1,1.0,-421.0,-413.0,100663296.0,6.0


In [34]:
df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)]['Hc_genMotherPDG'].value_counts()

-413.0       28986
 413.0       28848
-511.0       26068
 511.0       25892
-423.0        4454
 423.0        4435
-433.0        2123
 433.0        2097
-10431.0       855
 10431.0       847
-415.0         547
 415.0         539
-10413.0       279
 10413.0       236
 100443.0      166
 30443.0       164
-10411.0       159
 4212.0        149
 10411.0       142
 4222.0        138
-4212.0        129
 4112.0        126
-4114.0        119
-4222.0        113
 4114.0        112
-4112.0        106
 20433.0        97
-20433.0        93
 4214.0         88
 20443.0        79
-4214.0         77
 4224.0         57
-4224.0         55
-20413.0        40
 20413.0        31
 445.0           7
-10433.0         2
 10441.0         1
Name: Hc_genMotherPDG, dtype: int64

## check if category combinations make sense

In [35]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby( ["basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
groupsAllFSPs

,basf2_used,basf2_Bsig,basf2_X,count
0,0.0,0.0,0.0,1412695
1,0.0,1.0,0.0,6
2,1.0,0.0,0.0,451361
3,1.0,0.0,1.0,1150224
4,1.0,1.0,0.0,838761


## delete the H_c FSPs (basf2_used==1 & basf2_Bsig==0 & basf2_X==0)

In [36]:
df_FSPs = df_FSPs[~((df_FSPs["basf2_used"]== 1) & (df_FSPs["basf2_Bsig"]== 0) & (df_FSPs["basf2_X"]== 0))]

In [37]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby( ["basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
groupsAllFSPs

,basf2_used,basf2_Bsig,basf2_X,count
0,0.0,0.0,0.0,1412695
1,0.0,1.0,0.0,6
2,1.0,0.0,1.0,1150224
3,1.0,1.0,0.0,838761


## keep only the ones where Hc genMotherPDG is 511

In [38]:
df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)]['Hc_genMotherPDG'].value_counts()

-413.0       28986
 413.0       28848
-511.0       26068
 511.0       25892
-423.0        4454
 423.0        4435
-433.0        2123
 433.0        2097
-10431.0       855
 10431.0       847
-415.0         547
 415.0         539
-10413.0       279
 10413.0       236
 100443.0      166
 30443.0       164
-10411.0       159
 4212.0        149
 10411.0       142
 4222.0        138
-4212.0        129
 4112.0        126
-4114.0        119
-4222.0        113
 4114.0        112
-4112.0        106
 20433.0        97
-20433.0        93
 4214.0         88
 20443.0        79
-4214.0         77
 4224.0         57
-4224.0         55
-20413.0        40
 20413.0        31
 445.0           7
-10433.0         2
 10441.0         1
Name: Hc_genMotherPDG, dtype: int64

In [39]:
df_Y4S = df_Y4S[(df_Y4S['Hc_genMotherPDG'] == 511.0) | (df_Y4S['Hc_genMotherPDG'] == -511.0)]

df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]

In [40]:
print(df_Y4S.shape[0])
print(df_FSPs.shape[0])

51960
1362611


## is B-sig really the B->D*lnu?

In [41]:
groupsBsig = pd.DataFrame({'count' : df_Y4S.groupby( ["Hc_motherUniqParID","Bsig_uniqParID"] ).size()}).reset_index()
groupsBsig

,Hc_motherUniqParID,Bsig_uniqParID,count
0,83886081.0,83886081.0,455
1,83886081.0,83886082.0,25440
2,83886082.0,83886081.0,25927
3,83886082.0,83886082.0,138


In [42]:
df_Y4S = df_Y4S[~(df_Y4S["Hc_motherUniqParID"] == df_Y4S["Bsig_uniqParID"])]
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]

## investigate the neutrals cut w/ goodBelleGamma

In [43]:
cuts = [0.050,0.100]

num_all_bg_neutrals = df_FSPs[((df_FSPs["PDG"]==22.) & (df_FSPs["B_ID"]==0.))].shape[0] 

for cut in cuts:
    num_bg_cutNeutrals = df_FSPs[((df_FSPs["p"]<cut) & (df_FSPs["PDG"]==22.) & (df_FSPs["B_ID"]==0.))].shape[0] 
    num_all_cutNeutrals = df_FSPs[((df_FSPs["p"]<cut) & (df_FSPs["PDG"]==22.))].shape[0]
    print("for cut p >",cut)
    print("this fraction of cut outs is bg:", round(num_bg_cutNeutrals/num_all_cutNeutrals,3))
    print("this fraction of bg is cut out:", round(num_bg_cutNeutrals/num_all_bg_neutrals,3))
    
num_bg_cutNeutrals = df_FSPs[((df_FSPs["goodBelleGamma"]!=1.) & (df_FSPs["PDG"]==22.) & (df_FSPs["B_ID"]==0.))].shape[0] 
num_all_cutNeutrals = df_FSPs[((df_FSPs["goodBelleGamma"]!=1.) & (df_FSPs["PDG"]==22.))].shape[0]
print("for cut goodBelleGamma")
print("this fraction of cut outs is bg:", round(num_bg_cutNeutrals/num_all_cutNeutrals,3))
print("this fraction of bg is cut out:", round(num_bg_cutNeutrals/num_all_bg_neutrals,3))

for cut p > 0.05
this fraction of cut outs is bg: 0.908
this fraction of bg is cut out: 0.633
for cut p > 0.1
this fraction of cut outs is bg: 0.802
this fraction of bg is cut out: 0.857
for cut goodBelleGamma
this fraction of cut outs is bg: 0.874
this fraction of bg is cut out: 0.754


In [44]:
num_all_cutNeutrals # this is how many gamma will be cut with goodBelleGamma

464552

## deeper look at goodBelleGamma

In [45]:
df_FSPs["bg"] = (df_FSPs["B_ID"] == 0.0)

In [46]:
groups_goodGamma = pd.DataFrame({'count' : df_FSPs.groupby( ["goodBelleGamma","basf2_used","bg"] ).size()}).reset_index()
groups_goodGamma

,goodBelleGamma,basf2_used,bg,count
0,0.0,0.0,False,30400
1,0.0,0.0,True,357292
2,0.0,1.0,False,28174
3,0.0,1.0,True,48686
4,1.0,0.0,False,59496
5,1.0,0.0,True,57104
6,1.0,1.0,False,283035
7,1.0,1.0,True,75383


In [47]:
gammasFSPs = df_FSPs[df_FSPs["PDG"] == 22.0]

groups_goodGamma = pd.DataFrame({'count' : gammasFSPs.groupby( ["genMothPDG_0"] ).size()}).reset_index()
groups_goodGamma.sort_values("count",ascending=False)

,genMothPDG_0,count
38,0.0,538465
43,111.0,268364
34,-211.0,18340
29,-321.0,15371
54,321.0,14996
...,...,...
53,315.0,1
73,3114.0,1
68,2112.0,1
76,3222.0,1


In [48]:
groups_goodGamma = pd.DataFrame({'count' : df_FSPs.groupby( ["goodBelleGamma","bg"] ).size()}).reset_index()
groups_goodGamma

,goodBelleGamma,bg,count
0,0.0,False,58574
1,0.0,True,405978
2,1.0,False,342531
3,1.0,True,132487


In [49]:
df_FSPs[(df_FSPs["goodBelleGamma"] == 0.0) & (df_FSPs["basf2_used"] == 1.0)]["B_ID"].value_counts()

0           48686
83886082    14122
83886081    14052
Name: B_ID, dtype: int64

In [50]:
df_FSPs[#(df_FSPs["goodBelleGamma"] == 0.0) & (df_FSPs["basf2_used"] == 1.0) &
        (df_FSPs["mcPDG"].isna() == False)][['E','mcPDG',
'B_ID',"basf2_used","basf2_Bsig","basf2_X",'genMothPDG_0','genMothPDG_1',                                                                              
'genMothPDG_2','genMothPDG_3','genMothPDG_4','genMothPDG_5','genMothPDG_6',
'genMothPDG_7','genMothPDG_8','genMothPDG_9']][:15]


,E,mcPDG,B_ID,basf2_used,basf2_Bsig,basf2_X,genMothPDG_0,genMothPDG_1,genMothPDG_2,genMothPDG_3,genMothPDG_4,genMothPDG_5,genMothPDG_6,genMothPDG_7,genMothPDG_8,genMothPDG_9
173422,0.516328,211.0,83886081,1.0,0.0,1.0,-431.0,-511.0,300553.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173434,0.423463,211.0,83886081,1.0,0.0,1.0,-421.0,-413.0,511.0,300553.0,0.0,0.0,0.0,0.0,0.0,0.0
173432,0.245971,211.0,83886082,1.0,1.0,0.0,-421.0,-423.0,-511.0,300553.0,0.0,0.0,0.0,0.0,0.0,0.0
173430,0.769378,-211.0,83886081,1.0,0.0,1.0,-431.0,-511.0,300553.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173429,0.205041,-211.0,83886081,1.0,1.0,0.0,-431.0,-511.0,300553.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173427,1.134902,-211.0,83886082,1.0,1.0,0.0,-421.0,-413.0,511.0,300553.0,0.0,0.0,0.0,0.0,0.0,0.0
173439,0.211869,-211.0,83886081,1.0,1.0,0.0,-413.0,511.0,300553.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173440,0.386843,-211.0,83886082,1.0,0.0,1.0,113.0,-511.0,300553.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173454,2.050567,-211.0,83886082,1.0,1.0,0.0,-421.0,-413.0,511.0,300553.0,0.0,0.0,0.0,0.0,0.0,0.0
173452,1.207510,-211.0,83886081,1.0,0.0,1.0,-411.0,511.0,300553.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
df_FSPs[(df_FSPs["goodBelleGamma"] == 0.0) &
        (df_FSPs["mcPDG"].isna() == False)]["mcPDG"].value_counts()

 22.0     49391
-11.0      4180
 11.0      1840
-13.0      1127
-211.0      848
 13.0       635
 211.0      548
-321.0        2
 130.0        2
 321.0        1
Name: mcPDG, dtype: int64

## cut on goodBelleGamma

In [52]:
df_FSPs.shape[0]

1350188

In [53]:
df_FSPs = df_FSPs[~((df_FSPs["goodBelleGamma"]!=1.) & (df_FSPs["PDG"]==22.))]

In [54]:
df_FSPs.shape[0]

885636

## delete delete_top_numFSP_evts

In [55]:
numberFSPsEvts = pd.DataFrame({'count' : df_FSPs.groupby(["__event__"]).size()}).reset_index()
numEvtsBeforeCut = df_Y4S.shape[0]

In [56]:
# print first to adjust percentile used
print("original max FSPs:",numberFSPsEvts["count"].max())
for percentile_lvl in [0.995,0.996,0.997,0.998,0.999]:
    print(percentile_lvl)
    percentile = numberFSPsEvts["count"].quantile(q=percentile_lvl)
    cut_numberFSPsEvts = numberFSPsEvts[numberFSPsEvts["count"] < percentile]
    
    #TMP_df_FSPs = df_FSPs[df_FSPs['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    TMP_df_Y4S = df_Y4S[df_Y4S['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    
    print("would delete", round(100 - (TMP_df_Y4S.shape[0]/df_Y4S.shape[0])*100,2),"% of evts")
    print("new max FSPs:",cut_numberFSPsEvts["count"].max())

original max FSPs: 40
0.995
would delete 0.51 % of evts
new max FSPs: 28
0.996
would delete 0.51 % of evts
new max FSPs: 28
0.997
would delete 0.51 % of evts
new max FSPs: 28
0.998
would delete 0.27 % of evts
new max FSPs: 29
0.999
would delete 0.12 % of evts
new max FSPs: 31


In [57]:
print(df_FSPs.shape[0])
print(df_Y4S.shape[0])


if delete_top_numFSP_evts:
    delete_percentileLvl = 0.998
    print("used delete_percentileLvl:",delete_percentileLvl)
    
    
    print(numberFSPsEvts["count"].describe())
    
    percentile = numberFSPsEvts["count"].quantile(q=delete_percentileLvl)
    cut_numberFSPsEvts = numberFSPsEvts[numberFSPsEvts["count"] < percentile]
    
    df_FSPs = df_FSPs[df_FSPs['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    df_Y4S = df_Y4S[df_Y4S['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    
print(df_FSPs.shape[0])
print(df_Y4S.shape[0])
print("deleted", round(100 - (df_Y4S.shape[0]//numEvtsBeforeCut)*100,2),"% of events")
numberFSPsEvts = pd.DataFrame({'count' : df_FSPs.groupby(["__event__"]).size()}).reset_index()
print(numberFSPsEvts["count"].describe())


885636
51367
used delete_percentileLvl: 0.998
count    51313.000000
mean        17.259486
std          4.121096
min          5.000000
25%         14.000000
50%         17.000000
75%         20.000000
max         40.000000
Name: count, dtype: float64
881327
51230
deleted 100 % of events
count    51177.000000
mean        17.221154
std          4.057468
min          5.000000
25%         14.000000
50%         17.000000
75%         20.000000
max         29.000000
Name: count, dtype: float64


## cut on momenta for neutrals, p>50MeV

In [58]:
#df_FSPs_final = df_FSPs_final[~((df_FSPs_final["p"]<0.050) & (df_FSPs_final["PDG"]==22.))]

In [59]:
#print(df_FSPs_final.shape[0])
#print(df_Y4S_final.shape[0])

## impute the nan values of variables

In [60]:
## data preprocessing, dealing with NaN
imputelist = [["pionID",-1.],["kaonID",-1.],["electronID",-1.],["muonID",-1.]
              ,["clusterReg",-1.],["clusterE9E21",-1.]
              ]
for impute in imputelist:
    column_name = impute[0]
    impute_val = impute[1]
    mask = df_FSPs[column_name].isna() == 1
    df_FSPs.loc[mask, column_name] = impute_val
    print(df_FSPs[df_FSPs[column_name].isna() == True].shape[0])

0
0
0
0
0
0


## save the dfs as they will be used for data prod

In [61]:
df_FSPs_final = df_FSPs
df_Y4S_final = df_Y4S

In [62]:
root_path

'/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_CMvars_run1/'

In [63]:
df_FSPs_final.to_csv(root_path + "final_df_FSPs_preProcessed_SHR.csv")
df_Y4S_final.to_csv(root_path + "final_df_Y4S_preProcessed_SHR.csv")

# start of NN data creation

In [64]:
numFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["__event__"] ).size()}).reset_index()

minFSPs = numFSPs["count"].min()
maxFSPs = numFSPs["count"].max()
print("minFSPs:",minFSPs)
print("maxFSPs:",maxFSPs,'\n')
print("num Events:",numFSPs.shape[0],'\n')

df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')

minFSPs: 5
maxFSPs: 29 

num Events: 51177 



In [66]:
variable_combinations = [
    [[    "charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID"],"SUPA_no_M"],
    [["M",         "dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID"],"SUPA_no_charge"],
    [["M","charge",     "dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID"],"SUPA_no_dr"],
    [["M","charge","dr",     "clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID"],"SUPA_no_dz"],
    [["M","charge","dr","dz",             "clusterE9E21","pionID","kaonID","electronID","muonID"],"SUPA_no_clReg"],
    [["M","charge","dr","dz","clusterReg",               "pionID","kaonID","electronID","muonID"],"SUPA_no_E9E21"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21",         "kaonID","electronID","muonID"],"SUPA_no_pID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID",         "electronID","muonID"],"SUPA_no_kID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID",             "muonID"],"SUPA_no_eID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID"         ],"SUPA_no_muID"]
]

In [69]:
variable_combinations = [
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID"],"SUPA_allExtras"],
    [["M","charge","pionID","kaonID","electronID","muonID"],"SUPA_IDchargeM"],
    [["M","charge"],"SUPA_chargeM"],
    [["charge","pionID","kaonID","electronID","muonID"],"SUPA_chargeIDs"],
    [["charge","pionID"],"SUPA_chargePionID"],
    [[],"SUPA_only4mom"],
    [["charge"],"SUPA_charge"]
]

In [67]:
for var_comb in variable_combinations:
    ablationSub = var_comb[1]
    data_dir = Path(nfs_path + "data/" + data_subdir + root_subdir + ablationSub + "/")    
    if save_data:
        data_dir.mkdir(parents=True, exist_ok=True)
    print("Will save data to:", data_dir,'is', save_data ,'\n')


    #minFSPs = 5
    for num_FSPs_toData in range(minFSPs, maxFSPs+1):
        df_num_subset = df_FSPs_final.copy()
        df_num_subset = df_num_subset[df_num_subset['numFSPs'] == num_FSPs_toData]


        numEvents = df_num_subset.__event__.nunique()
        print("numEvents:",numEvents)
        print("num_FSPs_toData:",num_FSPs_toData)  
        if numEvents == 0:
            print("skipped because empty \n")
            continue

        if numEvents < 10:
            print("skipped because <10 events \n")
            continue

        num_features = 4 + len(var_comb[0])
        leaves = np.zeros((numEvents, num_FSPs_toData,  num_features))  
        SA_target =  np.zeros((numEvents, num_FSPs_toData))
        global_tag = np.chararray((numEvents, num_FSPs_toData + 1), itemsize=30)

        event_list = df_num_subset[df_num_subset["numFSPs"] == num_FSPs_toData]["__event__"].unique()
        #print("len(event_list):",len(event_list))
        for i in range(numEvents):

            event_iter = event_list[i]

            global_tag_masterInfo = "evt" + str(event_iter)
            global_tag[i,-1] = global_tag_masterInfo
            #print("global_tag[i,-1]:",global_tag[i,-1])
            #print("i:",i,"event_iter:",event_iter)

            event_df = df_num_subset[df_num_subset.__event__ == event_iter]

            for j in range(num_FSPs_toData):
                #print("numParticle:",j)
                particle = event_df.iloc[j]

                #print(particle["mcPDG"],particle["px"],particle["py"],particle["pz"],particle["E"])
                leaves[i,j,0] = particle["px"]
                leaves[i,j,1] = particle["py"]
                leaves[i,j,2] = particle["pz"]
                leaves[i,j,3] = particle["E"]
                
                for k in range(len(var_comb[0])):
                    leav_idx = k + 4
                    leaves[i,j,leav_idx] = particle[var_comb[0][k]]
                    

                basf2_usage = "basf2_NONE"
                if particle["basf2_Bsig"] == 1.0:
                    basf2_usage = "basf2_Bsig"
                elif particle["basf2_X"] == 1.0:
                    basf2_usage = "basf2_X"
                elif particle["basf2_used"] == 0:
                    basf2_usage = "basf2_bg"

                global_tag_Info = str((particle["mcPDG"])) 
                global_tag_Info += "_" + basf2_usage
                global_tag[i,j] = global_tag_Info

                label = -10 # error code if assignment fails
                B_tag_uniqID = -10 # error code if assignment fails     

                B_sig_uniqID = df_Y4S_final[df_Y4S_final["__event__"] == event_iter].iloc[0]['Bsig_uniqParID']
                if B_sig_uniqID == 83886082.0:
                    B_tag_uniqID = 83886081.0
                elif B_sig_uniqID == 83886081.0:
                    B_tag_uniqID = 83886082.0

                if particle["B_ID"] == B_tag_uniqID:
                    label = 1 # particle belongs to X (MC truth)
                elif particle["B_ID"] == B_sig_uniqID:
                    label = 2 # particle belongs to Bsig (MC truth)
                elif particle["B_ID"] == 0:
                    label = 0 # background (MC truth)


                SA_target[i,j] = label

            del event_df


        # shuffle the data    
        for idx in np.arange(leaves.shape[0]):   # arange is like range but gives ndarray instead of list
            perms = np.random.permutation(leaves.shape[1])

            leaves[idx,:] = leaves[idx,perms]
            SA_target[idx,:] = SA_target[idx,perms]
            global_tag[idx,0:-1] = global_tag[idx,perms]





        #print(global_tag)
        train_ratio = 0.8
        validation_ratio = 0.2
        #test_ratio = 0.06

        print("leaves.shape:",leaves.shape)
        print("SA_target.shape:",SA_target.shape)
        print("global_tag.shape:",global_tag.shape)


        print("leaves[0]:",leaves[0])
        print("SA_target[0]:",SA_target[0])
        print("global_tag[0]:",global_tag[0])

        x=leaves
        y=SA_target
        z=global_tag

        x_train, x_val, y_train, y_val, z_train, z_val = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
        #x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
        #x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(x_test, y_test, z_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=False) 

        if save_data==True:
            np.save(data_dir / "leaves_train_FSP{}.npy".format(num_FSPs_toData), x_train)
            np.save(data_dir / "is_left_arr_train_FSP{}.npy".format(num_FSPs_toData), y_train)
            np.save(data_dir / "global_tag_train_FSP{}.npy".format(num_FSPs_toData), z_train)

            np.save(data_dir / "leaves_val_FSP{}.npy".format(num_FSPs_toData), x_val)
            np.save(data_dir / "is_left_arr_val_FSP{}.npy".format(num_FSPs_toData), y_val)
            np.save(data_dir / "global_tag_val_FSP{}.npy".format(num_FSPs_toData), z_val)

            #np.save(data_dir / "leaves_test_FSP{}.npy".format(num_FSPs_toData), x_test)
            #np.save(data_dir / "is_left_arr_test_FSP{}.npy".format(num_FSPs_toData), y_test)
            #np.save(data_dir / "global_tag_test_FSP{}.npy".format(num_FSPs_toData), z_test)


        print("")
        #del df_num_subset


        del df_num_subset


Will save data to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/data/FULLCMrun_Dstlnu_SHR_BsX_Ablations/SHR_CMvars_run1/SUPA_no_M is True 

numEvents: 7
num_FSPs_toData: 5
skipped because <10 events 

numEvents: 42
num_FSPs_toData: 6
leaves.shape: (42, 6, 13)
SA_target.shape: (42, 6)
global_tag.shape: (42, 7)
leaves[0]: [[-1.55037153e+00  9.40281689e-01  1.58184779e+00  2.41029124e+00
   1.00000000e+00  1.07741070e-03 -3.78275517e-02  2.00000000e+00
   1.00000000e+00  8.17980131e-01  1.66051148e-06  8.29847193e-10
   1.82018208e-01]
 [ 1.55090792e-02 -1.17995381e-01 -7.34342262e-02  1.97575099e-01
   1.00000000e+00  2.47451864e-02 -4.31642743e-02 -1.00000000e+00
  -1.00000000e+00  9.80899162e-01  4.09096633e-12  3.72910862e-03
   1.53717296e-02]
 [ 3.87008600e-02  3.45234238e-02 -9.37034562e-03  5.27012972e-02
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00]
 [-2.1586330


numEvents: 1174
num_FSPs_toData: 10
leaves.shape: (1174, 10, 13)
SA_target.shape: (1174, 10)
global_tag.shape: (1174, 11)
leaves[0]: [[ 6.57277927e-02 -3.17256488e-02  5.99200539e-02  9.44302514e-02
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00]
 [ 4.67068613e-01 -4.92854506e-01  1.02814054e+00  1.23212484e+00
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   9.86328125e-01 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00]
 [-5.86255416e-02 -5.10016978e-02  3.93570326e-02  1.64520503e-01
   1.00000000e+00  3.33056072e-01  4.20142434e-01 -1.00000000e+00
  -1.00000000e+00  9.65222742e-01  5.53859305e-07  1.38918994e-30
   3.47765954e-02]
 [-6.22908771e-01  5.27401805e-01  9.89789665e-02  8.33933288e-01
   1.00000000e+00  6.72435571e-03 -1.53001211e-02  2.00000000e+00
   9.60937500e-01  9.99829534e-01  1.13371533e-09  3.27105618e-08
   1.70432296e-04


numEvents: 3138
num_FSPs_toData: 13
leaves.shape: (3138, 13, 13)
SA_target.shape: (3138, 13)
global_tag.shape: (3138, 14)
leaves[0]: [[ 6.27322376e-01 -8.61062586e-01  4.24446702e-01  1.14678557e+00
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   9.77539062e-01 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00]
 [-7.95749575e-03 -1.02617994e-01 -1.90024730e-04  1.02926238e-01
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   9.88281250e-01 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00]
 [ 3.42271477e-02 -4.38979834e-01 -2.49170169e-01  5.05925455e-01
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   9.39453125e-01 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00]
 [ 3.03403914e-01 -1.44165441e-01  4.08514500e-01  5.46993234e-01
   1.00000000e+00  3.79256051e-03  1.05933430e-01  2.00000000e+00
   7.46093750e-01  9.99991121e-01  7.04581297e-52  2.66672874e-08
   8.85227970e-06


numEvents: 4690
num_FSPs_toData: 16
leaves.shape: (4690, 16, 13)
SA_target.shape: (4690, 16)
global_tag.shape: (4690, 17)
leaves[0]: [[ 1.03565775e-01  1.14231981e-01  6.79248646e-02  1.68489176e-01
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   9.49218750e-01 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00]
 [-7.96577409e-02  1.92935586e-01 -1.54564790e-02  2.09304560e-01
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   9.94140625e-01 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00]
 [-2.36037802e-02 -5.49455546e-02  2.01969463e-02  6.31194823e-02
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00]
 [ 6.41710758e-02  1.10448472e-01 -9.49195698e-02  1.59143070e-01
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   9.64843750e-01 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00


numEvents: 4780
num_FSPs_toData: 18
leaves.shape: (4780, 18, 13)
SA_target.shape: (4780, 18)
global_tag.shape: (4780, 19)
leaves[0]: [[ 2.07457945e-001  1.72078148e-001 -6.18561469e-002  3.09767272e-001
  -1.00000000e+000  1.12224802e-002  1.93592212e-002 -1.00000000e+000
  -1.00000000e+000  1.39612183e-015  7.17437150e-137  2.99535018e-060
   1.00000000e+000]
 [ 5.66750690e-002  9.45582837e-002  1.29282728e-001  1.69903962e-001
   0.00000000e+000  1.18686603e-009  2.23517418e-010  2.00000000e+000
   8.30078125e-001 -1.00000000e+000 -1.00000000e+000 -1.00000000e+000
  -1.00000000e+000]
 [-9.46453512e-002  4.64304030e-001  9.67042819e-002  5.03356320e-001
  -1.00000000e+000  1.59023997e-003  1.57445191e-002  2.00000000e+000
   9.25781250e-001  9.90319531e-001  8.40005514e-008  3.61789397e-008
   9.68034907e-003]
 [ 3.10274333e-001 -1.61027825e+000  2.45894253e-001  1.66159378e+000
   1.00000000e+000  2.94048196e-003  5.59417610e-003  2.00000000e+000
   9.92187500e-001  1.38615420e-003 

...


numEvents: 3307
num_FSPs_toData: 21
leaves.shape: (3307, 21, 13)
SA_target.shape: (3307, 21)
global_tag.shape: (3307, 22)
leaves[0]: [[ 2.30094455e-02  6.08646125e-02 -1.47445295e-02  6.67183392e-02
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00]
 [ 9.47609782e-01  1.39989936e+00 -2.21659988e-01  1.70493869e+00
   5.10998943e-04  1.00000000e+00  2.45675247e-05  4.70147461e-03
   2.00000000e+00  9.89257812e-01  1.67261384e-04  1.57201431e-17
   9.99832739e-01]
 [-3.30370635e-01  1.94189310e-01  7.89795935e-01  8.88882307e-01
   1.39570385e-01 -1.00000000e+00  9.01412896e-03  1.05957690e-02
   1.00000000e+00  9.95117188e-01  9.32885668e-02  1.53895466e-01
   1.17333120e-03]
 [-6.52725324e-02 -3.67194563e-02  8.46466646e-02  1.79593352e-01
   1.39570385e-01 -1.00000000e+00  3.07291461e-02 -1.01365071e-02
  -1.00000000e+00 -1.00000000e+00  9.94723654e-01  1.14275886e-08
   2.07832194e-09


numEvents: 1774
num_FSPs_toData: 23
leaves.shape: (1774, 23, 13)
SA_target.shape: (1774, 23)
global_tag.shape: (1774, 24)
leaves[0]: [[ 7.38859698e-02  2.25548178e-01 -3.40973102e-02  2.39778530e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  9.93164062e-01 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00]
 [ 1.40043050e-01  1.73652530e-01  3.62983763e-01  4.48335100e-01
   1.39570385e-01 -1.00000000e+00  1.39997484e-02  1.47655753e-03
   1.00000000e+00  9.62890625e-01  3.71771370e-31  1.00000000e+00
   1.82981114e-15]
 [-2.02710658e-01 -8.63474682e-02 -6.49423301e-02  2.68784105e-01
   1.39570385e-01  1.00000000e+00  1.28525487e-02 -3.65080026e-03
  -1.00000000e+00 -1.00000000e+00  4.84986517e-01  3.42337625e-21
   3.68300419e-01]
 [-9.35334712e-02  6.18762821e-02 -3.32737826e-02  1.16980037e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00


numEvents: 818
num_FSPs_toData: 25
leaves.shape: (818, 25, 13)
SA_target.shape: (818, 25)
global_tag.shape: (818, 26)
leaves[0]: [[ 1.82392150e-001 -1.49327368e-001  3.99071842e-001  4.84049364e-001
   1.39570385e-001  1.00000000e+000  5.40225096e-003 -1.43447131e-002
   1.00000000e+000  8.35937500e-001  2.44148337e-001  5.58777972e-001
   5.51273405e-005]
 [-7.47205079e-001 -7.36283213e-002  4.76571685e-003  7.63700965e-001
   1.39570385e-001 -1.00000000e+000  5.26226207e-004 -2.42950414e-002
   2.00000000e+000  7.76367188e-001  9.94139721e-001  1.63396393e-009
   3.54011263e-005]
 [ 2.43699029e-001  1.35272458e-001  2.21257329e-001  3.82259798e-001
   1.39570385e-001 -1.00000000e+000  1.70190289e-002  1.42644356e-002
  -1.00000000e+000 -1.00000000e+000  6.91088795e-081  8.15195651e-256
   4.55028525e-007]
 [ 1.93854123e-001 -1.93560496e-001  1.75702348e-001  5.91298040e-001
   4.93676990e-001 -1.00000000e+000  1.38231762e-002 -2.11781063e-002
   1.00000000e+000  9.37500000e-001  5.1


numEvents: 333
num_FSPs_toData: 27
leaves.shape: (333, 27, 13)
SA_target.shape: (333, 27)
global_tag.shape: (333, 28)
leaves[0]: [[ 5.96398525e-02  3.86205614e-02  4.73510195e-03  7.12101184e-02
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  9.47265625e-01 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00]
 [-3.91403027e-02  1.18431244e-02  1.44927591e-01  1.50586286e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   1.00000000e+00  1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00]
 [ 2.58126557e-01  2.80684859e-01  3.55514050e-01  7.17997576e-01
   4.93676990e-01 -1.00000000e+00  3.58962315e-03  8.77335281e-02
   2.00000000e+00  1.00000000e+00  1.23925223e-07  9.99999846e-01
   2.17022970e-16]
 [-1.73724238e-02 -4.78420891e-02  3.67896818e-02  6.28024465e-02
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  9.93164062e-01 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00]
 [


numEvents: 127
num_FSPs_toData: 29
leaves.shape: (127, 29, 13)
SA_target.shape: (127, 29)
global_tag.shape: (127, 30)
leaves[0]: [[ 9.09427032e-002  2.66557420e-003 -1.14856465e-002  9.17038746e-002
   0.00000000e+000  0.00000000e+000  1.18686603e-009  2.23517418e-010
   2.00000000e+000  9.93164062e-001 -1.00000000e+000 -1.00000000e+000
  -1.00000000e+000]
 [ 3.82777266e-002  8.61745253e-002  1.22539073e-001  1.54619073e-001
   0.00000000e+000  0.00000000e+000  1.18686603e-009  2.23517418e-010
   2.00000000e+000  1.00000000e+000 -1.00000000e+000 -1.00000000e+000
  -1.00000000e+000]
 [-3.20656478e-001  6.20187121e-003 -2.06253231e-001  4.06053356e-001
   1.39570385e-001 -1.00000000e+000  6.76921343e-003 -3.82496438e-002
   1.30000000e+001  8.40820312e-001  5.63790991e-001  2.12104910e-012
   1.07076687e-002]
 [ 3.10195982e-002 -9.82017368e-002  1.57459393e-001  1.88146903e-001
   0.00000000e+000  0.00000000e+000  1.18686603e-009  2.23517418e-010
   2.00000000e+000  9.62890625e-001 -1.0

In [68]:
print("saving is done")
now = datetime.now()
print("time at end =", now)

saving is done
time at end = 2021-10-17 18:32:27.001961


In [ ]:
#why only one FSP in so many events?

In [59]:
df_num_subset = df_FSPs_final.copy()
df_num_subset = df_num_subset[df_num_subset['numFSPs'] == 1]

In [51]:
df_FSPs_final.keys()

Index(['Unnamed: 0', '__experiment__', '__run__', '__event__', '__candidate__',
       '__ncandidates__', '__weight__', 'basf2_X', 'basf2_used', 'basf2_Bsig',
       'isSignal', 'uniqueParticleIdentifier', 'mcErrors', 'mcPDG',
       'genMotherID', 'genMotherP', 'genMotherPDG', 'px', 'py', 'pz', 'pt',
       'p', 'E', 'kaonID', 'pionID', 'genMothPDG_0', 'genMothPDG_1',
       'genMothPDG_2', 'genMothPDG_3', 'genMothPDG_4', 'genMothPDG_5',
       'genMothPDG_6', 'genMothPDG_7', 'genMothPDG_8', 'genMothPDG_9',
       'genMotherID_0', 'genMotherID_1', 'genMotherID_2', 'genMotherID_3',
       'genMotherID_4', 'genMotherID_5', 'genMotherID_6', 'genMotherID_7',
       'genMotherID_8', 'genMotherID_9', 'mcMother0_uniqParID',
       'mcMother1_uniqParID', 'mcMother2_uniqParID', 'mcMother3_uniqParID',
       'mcMother4_uniqParID', 'mcMother5_uniqParID', 'mcMother6_uniqParID',
       'mcMother7_uniqParID', 'mcMother8_uniqParID', 'mcMother9_uniqParID',
       'PDG', 'B_ID', 'numFSPs'],
      dtyp

In [52]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
groupsAllFSPs

,basf2_used,basf2_Bsig,basf2_X,count
0,0.0,0.0,0.0,591463
1,0.0,1.0,0.0,1
2,1.0,0.0,1.0,479138
3,1.0,1.0,0.0,349332


In [3]:
fileY4S = uproot.open("/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_wChargeLessFiles_run1/DXtagDstl.root:variables")
df_Y4S = fileY4S.arrays(library="pd")

In [4]:
pions = uproot.open("/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_wChargeLessFiles_run1/pions.root:variables")
df_pions = pions.arrays(library="pd")

NameError: name 'event_df' is not defined

In [114]:
df_Y4S_final["Hc_mcPDG"].value_counts()

 411.0     12191
-411.0     12132
 413.0      2921
-421.0      2902
 421.0      2883
-413.0      2802
-431.0      1098
 431.0      1065
 443.0       831
 4122.0      154
-4122.0      137
 423.0        39
-423.0        36
Name: Hc_mcPDG, dtype: int64

In [6]:
df_Y4S[(df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)].shape[0]

11

In [69]:
for i in variable_combinations:
    print(i[1])

SUPA_no_M
SUPA_no_charge
SUPA_no_dr
SUPA_no_dz
SUPA_no_clReg
SUPA_no_E9E21
SUPA_no_pID
SUPA_no_kID
SUPA_no_eID
SUPA_no_muID
